# Machine Learning LAB 5: Random Forests

Course 2024/25: *F. Chiariotti*

The notebook contains a simple learning task over which we will implement a **RANDOM FOREST**.

Complete all the **required code sections**.

### IMPORTANT for the exam:

The functions you might be required to implement in the exam will have the same signature and parameters as the ones in the labs

## Classification of Stayed/Churned Customers

The Customer Churn table contains information on all 3,758 customers from a Telecommunications company in California in Q2 2022. Companies are naturally interested in churn, i.e., in which users are likely to switch to another company soon to get a better deal, and which are more loyal customers.

The dataset contains three features:
- **Tenure in Months**: Number of months the customer has stayed with the company
- **Monthly Charge**: The amount charged to the customer monthly
- **Age**: Customer's age

The aim of the task is to predict if a customer will churn or not based on the three features.

---

## Import all the necessary Python libraries and load the dataset

### The Dataset
The dataset is a `.csv` file containing three input features and a label. Here is an example of the first 4 rows of the dataset: 

<center>

Tenure in Months | Monthly Charge | Age | Customer Status |
| -----------------| ---------------|-----|-----------------|
| 9 | 65.6 | 37 | 0 |
| 9 | -4.0 | 46 | 0 |
| 4 | 73.9 | 50 | 1 |
| ... | ... | ... | ... |

</center>

Customer Status is 0 if the customer has stayed with the company and 1 if the customer has churned.

In [1]:
import numpy as np
import pandas as pd
import random as rnd
from matplotlib import pyplot as plt
from sklearn import linear_model, preprocessing
from sklearn.model_selection import train_test_split

np.random.seed(1)

def load_dataset(filename):
    data_train = pd.read_csv(filename)
    #permute the data
    data_train = data_train.sample(frac=1).reset_index(drop=True) # shuffle the data
    X = data_train.iloc[:, 0:3].values # Get first two columns as the input
    Y = data_train.iloc[:, 3].values # Get the third column as the label
    Y = 2*Y-1 # Make sure labels are -1 or 1 (0 --> -1, 1 --> 1)
    return X,Y

# Load the dataset
X, Y = load_dataset('data/telecom_customer_churn_cleaned.csv')

## Divide the data into training and test sets

We are going to differentiate (classify) between **class "1" (churned)** and **class "-1" (stayed)**

In [2]:
# Compute the splits
m_training = int(0.75*X.shape[0])

# m_test is the number of samples in the test set (total-training)
m_test =  X.shape[0] - m_training
X_training =  X[:m_training]
Y_training =  Y[:m_training]
X_test =   X[m_training:]
Y_test =  Y[m_training:]

print("Number of samples in the train set:", X_training.shape[0])
print("Number of samples in the test set:", X_test.shape[0])
print("Number of churned users in test:", np.sum(Y_test==-1))
print("Number of loyal users in test:", np.sum(Y_test==1))

# Standardize the input matrix
# The transformation is computed on training data and then used on all the 3 sets
scaler = preprocessing.StandardScaler().fit(X_training) 

np.set_printoptions(suppress=True) # sets to zero floating point numbers < min_float_eps
X_training =  scaler.transform(X_training)
print ("Mean of the training input data:", X_training.mean(axis=0))
print ("Std of the training input data:",X_training.std(axis=0))

X_test =  scaler.transform(X_test)
print ("Mean of the test input data:", X_test.mean(axis=0))
print ("Std of the test input data:", X_test.std(axis=0))
print(Y_training)

Number of samples in the train set: 2817
Number of samples in the test set: 940
Number of churned users in test: 479
Number of loyal users in test: 461
Mean of the training input data: [-0.  0. -0.]
Std of the training input data: [1. 1. 1.]
Mean of the test input data: [0.0575483  0.05550169 0.0073833 ]
Std of the test input data: [0.98593187 0.97629659 1.00427583]
[-1  1  1 ...  1 -1 -1]


We will use **homogeneous coordinates** to describe all the coefficients of the model.

_Hint:_ The conversion can be performed with the function $hstack$ in $numpy$.


**^**

**l**

**l**

**THIS CELL IS A TYPO (copied-pasted without check) - IGNORE IT** No point in adding a coordinate which is identical over all points, if you are doing a tree classifier!!

## Decision tree

Now **complete** the class *Tree* and all auxiliary functions. <br>

The input parameters to pass to the *id3_training* function are:
- $X$: the matrix of input features, one row for each sample
- $Y$: the vector of labels for the input features matrix X
- $max\_depth$: the maximum depth of the tree

*My notes:*

1. *class Tree is actually a NODE object. If the node is internal, it will contain a list of nodes making its left subtree, and a list of nodes making the right subtree (self.left and self.right attributes, respectively). Each node in the subtrees is an object of class Tree by itself.*

2. *Entropy function takes as inputs the lists of **labels Y**  (of points in the left subtree and of points in the right subtree). It is a static method, so you call it as Tree.entropy(). 

4. Careful to always include a `return` in the recursive function `self.classify()`. Also, remember to assign the attributes self.idx and self.thresh to each Tree during the training.

In [3]:
class Tree:

    def __init__(self):
        self.idx = -1    # The index of the feature over which you split (no split: -1)
        self.thresh = 0  # The threshold value over which you split (<=: left, >: right)
        self.leaf = 0    # 1 if it is a leaf of class 1, -1 if it is a leaf of class -1, 0 if it is an internal node
        self.left = []   # Left subtree (empty if it is a leaf)
        self.right = []  # Right subtree (empty if it is a leaf)


    def entropy(left, right):
        H = 0
        tot_length = len(left) + len(right)
        left_prob = len(np.where(left > 0)[0]) / len(left)
        if (left_prob > 0):
            H -= len(left) * left_prob * np.log2(left_prob) / tot_length
        if (left_prob < 1):
            H -= len(left) * (1 - left_prob) * np.log2(1 - left_prob) / tot_length
        right_prob = len(np.where(right > 0)[0]) / len(right)
        if (right_prob > 0):
            H -= len(right) * right_prob * np.log2(right_prob) / tot_length
        if (right_prob < 1):
            H -= len(right) * (1 - right_prob) * np.log2(1 - right_prob) / tot_length
        return H

    def classify(self, x):
        ## TODO: classify the point x (easy for leaves, you have to go down the tree if the node is internal)
        if (self.leaf != 0):
            return self.leaf # return leaf value (+ or - 1) as the point label
        elif (x[self.idx] <= self.thresh):
            return self.left.classify(x) #  if not a leaf, pass it on to the left or right node TO CHECK AGAIN
        else:
            return self.right.classify(x)


    def id3_training(self, X, Y, max_depth, printing):
        # Check if the node is a leaf (all nodes have the same label)
        if (np.max(Y) - np.min(Y) < 1e-3):
            self.leaf = np.max(Y)
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (all labels are the same over ' + str(len(Y)) + ' points)')
            return
        # If the maximum depth is 0, the node must be a leaf!
        if (max_depth < 1):
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (maximum depth reached, ' + str(len(Y)) + ' points)')
            if (len(np.where(Y > 0)) > len(Y) / 2):
                self.leaf = 1
            else:
                self.leaf = -1
            return
        # Find the best split: iterate over features
        best_idx = -1
        best_thresh = -1
        best_entropy = 1e9
        ## TODO: Iterate over the features and threshold values! 
        for idx in range(X.shape[1]):
            x_values = np.unique(X[:, idx]) # sorted unique values
            thresholds = np.array([(x_values[i+1] + x_values[i])/2 for i in range(len(x_values) - 1)]) # mean values
            for t in thresholds:
                left_samples =  np.where(X[:, idx] <= t)[0] #returns indexes
                right_samples = np.where(X[:, idx] > t)[0]
                entropy_value = Tree.entropy(Y[left_samples], Y[right_samples])
                if entropy_value <= best_entropy:
                    best_idx = idx
                    best_thresh = t
                    best_entropy = entropy_value
        ####-------------------------------------------------------####
        if (best_idx == -1):
            # No valid features! The points are all identical
            #
            # my note: entropy always > 10e9 means either that all points have same label (but cannot be,
            # because we ruled out that possibility already) or that all points are the same point (so)
            # distribution is a dirac delta
            self.leaf = np.sign(np.sum(Y))
            if (self.leaf == 0):
                self.leaf = 1
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (all inputs are the same over ' + str(len(Y)) + ' points)')
            return
        left_samples = np.where(X[:, best_idx] <= best_thresh)[0]
        right_samples = np.where(X[:, best_idx] > best_thresh)[0]
        if (printing):
            print('Remaining depth: ' + str(max_depth) + ', splitting ' + str(len(Y)) + ' elements into ' + str(len(left_samples)) + ' and ' + str(len(right_samples)) + ' over feature ' + str(best_idx))
        ## TODO: run the next recursive step of ID3 over the left and right subtrees!
        # also, we need to set the class attributes self.idx and self.threshold, which are then used for PREDICTION
        self.idx = best_idx
        self.thresh = best_thresh
        self.left = Tree()
        self.right = Tree()
        self.left.id3_training(X[left_samples], Y[left_samples], max_depth - 1, printing)
        self.right.id3_training(X[right_samples], Y[right_samples], max_depth - 1, printing)




    def extra_training(self, X, Y, max_depth, printing):
        # Check if the node is a leaf (all nodes have the same label)
        if (np.max(Y) - np.min(Y) < 1e-3):
            self.leaf = np.max(Y)
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (all labels are the same over ' + str(len(Y)) + ' points)')
            return
        # If the maximum depth is 0, the node must be a leaf!
        if (max_depth < 1):
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (maximum depth reached, ' + str(len(Y)) + ' points)')
            if (len(np.where(Y > 0)) > len(Y) / 2):
                self.leaf = 1
            else:
                self.leaf = -1
            return
        # Find the best split: iterate over features
        best_idx = -1
        best_thresh = -1
        best_entropy = 1e9
        ## TODO: Iterate over the features (remember, the threshold value is random)! 
        for idx in range(X.shape[1]):
            x_vals = np.unique(X[:, idx])
            thresholds = np.array([(x_vals[i+1] + x_vals[i])/2 for i in range(len(x_vals) - 1)])
            if len(thresholds) > 0:
                t = np.random.choice(a = thresholds, size = 1)[0]
                left_samples =  np.where(X[:, idx] <= t)[0] #returns indexes
                right_samples = np.where(X[:, idx] > t)[0]
                entropy = Tree.entropy(left_samples, right_samples)
                if (entropy <= best_entropy):
                    best_idx = idx
                    best_entropy = entropy
                    best_thresh = t
        if (best_idx == -1):
            # No valid features! The points are all identical
            self.leaf = np.sign(np.sum(Y))
            if (self.leaf == 0):
                self.leaf = 1
            if (printing):
                print('Remaining depth: ' + str(max_depth) + ', leaf node (all inputs are the same over ' + str(len(Y)) + ' points)')
            return
        left_samples = np.where(X[:, best_idx] <= best_thresh)[0]
        right_samples = np.where(X[:, best_idx] > best_thresh)[0]
        if (printing):
            print('Remaining depth: ' + str(max_depth) + ', splitting ' + str(len(Y)) + ' elements into ' + str(len(left_samples)) + ' and ' + str(len(right_samples)) + ' over feature ' + str(best_idx))
        ## TODO: run the next recursive step of ID3 over the left and right subtrees!
        self.idx = best_idx
        self.thresh = best_thresh
        self.left = Tree()
        self.right = Tree()
        self.left.extra_training(X[left_samples], Y[left_samples], max_depth - 1, printing)
        self.right.extra_training(X[right_samples], Y[right_samples], max_depth - 1, printing)

Now we use the implementation to learn a model from the training data directly. Set a maximum depth of 12.

In [4]:
single_tree = Tree()
single_tree.id3_training(X_training, Y_training, 12, False)

train_loss = 0
for i in range(len(Y_training)):
    predicted = single_tree.classify(X_training[i, :])
    if (Y_training[i] != predicted):
        train_loss += 1 / len(Y_training)
print('Training loss: ' + str(train_loss))

test_loss = 0
for i in range(len(Y_test)):
    predicted = single_tree.classify(X_test[i, :])
    if (Y_test[i] != predicted):
        test_loss += 1 / len(Y_test)
print('Test loss: ' + str(test_loss))

Training loss: 0.19453319133830368
Test loss: 0.3255319148936155


This overfits a lot! Let's try to create a random forest

In [5]:
class Forest:

    def __init__(self, trees, n_features, n_samples, max_depth):
        self.forest = [] # list of trees
        self.features = n_features # my note: to be used in bag()
        self.max_depth = max_depth
        self.samples = n_samples  # my note: to be used in bag()
        for i in range(trees):
            self.forest.append(Tree())

    def classify(self, x):
        # TODO Classify the point through a majority vote
        votes = [forest_tree.classify(x) for forest_tree in self.forest]
        vote = np.sign(np.sum(votes))
        if vote == 0:
            vote = +1 # in case of ties, I return 1
        return vote

    def train(self, X, Y):
        for tree in self.forest:
            X_train, Y_train = self.bag(X, Y)
            tree.id3_training(X_train, Y_train, self.max_depth, False)
        
    def bag(self, X, Y):
        ## TODO: implement bagging! Sample data points with replacement
        chosen_idxs = np.random.choice(np.arange(X.shape[0]), size = self.samples, replace = True)
        if self.features == X.shape[1]:
            chosen_features = np.arange(self.features)
        else:
            chosen_features = np.random.choice(np.arange(X.shape[1]), size = self.features, replace = False)
        X_bag = X[chosen_idxs][:, chosen_features]
        Y_bag = Y[chosen_idxs]
        return X_bag, Y_bag

Let us create a forest with **400** trees, with a maximum depth of 12. We can train each tree on the same number of points as the training set (using **bagging** to add some randomness). Since we do not have that many features, **we keep all 3 features**.

In [6]:
forest = Forest(40, 3, X.shape[0], 12)
forest.train(X_training, Y_training)

test_loss = 0
for i in range(len(Y_test)):
    predicted = forest.classify(X_test[i, :])
    if (Y_test[i] * predicted <= 0):
        test_loss += 1 / len(Y_test)
print('Test loss: ' + str(test_loss))

Test loss: 0.2861702127659569


Now we can try to see the effect of using Extra Trees. 

In [7]:
class ExtraTrees:

    def __init__(self, trees, n_features, n_samples, max_depth):
        self.forest = []
        self.features = n_features
        self.max_depth = max_depth
        self.samples = n_samples
        for i in range(trees):
            self.forest.append(Tree())

    def classify(self, x):
        vote = 0
        for tree in self.forest:
            vote += tree.classify(x)
        return np.sign(vote)

    def train(self, X, Y):
        for tree in self.forest:
            X_train, Y_train = self.bag(X, Y)
            tree.extra_training(X_train, Y_train, self.max_depth, False)
        
    def bag(self, X, Y):
        features = X.shape[1]
        points = X.shape[0]
        # Bagging: sample with replacement
        bagged = rnd.choices(range(points), k = self.samples)
        X_bagged = X[bagged, :]
        # Remove features that are not part of the tree
        selected = rnd.sample(range(features), k = self.features)
        for i in range(features):
            if i not in selected:
                X_bagged[:, i] = 0
        return X_bagged, Y[bagged]

Let us create an Extra Trees Classifer with 1000 trees, with a maximum depth of 12. We can train each tree on the same number of points as the training set (using bagging to add some randomness). Since we do not have that many features, we keep all 3 features

In [8]:
extra = ExtraTrees(100, 3, X.shape[0], 12)
extra.train(X_training, Y_training)

test_loss = 0
for i in range(len(Y_test)):
    predicted = extra.classify(X_test[i, :])
    if (Y_test[i] * predicted <= 0):
        test_loss += 1 / len(Y_test)
print('Test loss: ' + str(test_loss))

Test loss: 0.25851063829787246
